In [7]:
import numpy as np
import pandas as pd
import collections
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as plt3d
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import LinearSVC, SVC

In [10]:
normalize([[115.9967,202.6615,44.2614]])

array([[0.4880623 , 0.85270907, 0.1862322 ]])

# Zadanie 4

In [ ]:
data = pd.read_csv("./131767-svm.txt", sep='\t')
data[:3]
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [ ]:
classes_count = collections.Counter(y)
class_0_prc = classes_count[0]/(classes_count[1]+classes_count[0])
class_1_prc = 1 - class_0_prc
print("0: ", class_0_prc)
print("1: ", class_1_prc)

In [ ]:
collections.Counter(X.dtypes)
# wszystkie atrybuty liczbowe

In [ ]:
mean_df = pd.DataFrame(X.mean())
std_df = pd.DataFrame(X.std())
min_df = pd.DataFrame(X.min())
max_df = pd.DataFrame(X.max())

In [ ]:
mean_std_df = mean_df.merge(std_df, left_index=True, right_index=True)
mean_std_min_df = mean_std_df.merge(min_df, left_index=True, right_index=True)
mean_std_min_max_df = mean_std_min_df.merge(max_df, left_index=True, right_index=True)

In [ ]:
mean_std_min_max_df.columns = ['mean', "std", 'min', 'max']
mean_std_min_max_df

In [ ]:
# Klasa większościowa to 0 ~74%. To byłaby trafność klasyfikatora Zero Rule.
y_pred = np.zeros(data.shape[0])
print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))
print("balanced_accuracy_score: ", balanced_accuracy_score(y, y_pred, adjusted=False))

In [ ]:
sensitivity = 0/(0+3897)
specificity = 11103/(11103+0)
G_mean = pow(sensitivity*specificity,0.5)
G_mean # 0

# Zadanie 5

In [ ]:
sca = StandardScaler()
X_std = sca.fit_transform(X)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_std)

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
plt.scatter(X_pca[y==0][:,0], X_pca[y==0][:,1], color='red', edgecolor='black')
plt.scatter(X_pca[y==1][:,0], X_pca[y==1][:,1], color='blue', edgecolor='yellow')

plt.savefig("./rysunki/PCA_2D.svg")
print(": ", sum(pca.explained_variance_ratio_))

In [ ]:
from matplotlib import interactive
%matplotlib qt  
# wx, gtk, osx, tk, empty uses default

pca = PCA(n_components=3, random_state=42)
X_pca = pca.fit_transform(X_std)[:15000]

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
ax = plt.axes(projection='3d')
y_5k = y[:15000]
ax.scatter3D(X_pca[y_5k==0][:,0], X_pca[y_5k==0][:,1], X_pca[y_5k==0][:,2], color='red', edgecolor='black')
ax.scatter3D(X_pca[y_5k==1][:,0], X_pca[y_5k==1][:,1], X_pca[y_5k==1][:,2], color='blue', edgecolor='yellow')
# ax.view_init(10, 0)
# plt.savefig("./rysunki/t.svg")
print(": ", sum(pca.explained_variance_ratio_))

# Zadanie 6

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# skf  = StratifiedKFold(n_splits=2, shuffle=True)

# X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
# y = np.array([0, 0, 1, 1])

# for train_index, test_index in skf.split(X, y):
#     print("TEST:", test_index , "TRAIN:", train_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

'''
Jeśli dane są posortowane w jakiś sposób to włączenie/wyłączenie mieszania może mieć duży wpływ na uzyskaną trafność.
Załóżmy, iż mamy 100 przypadków uczących i używamy StratifiedFold z 5 foldami. Każdy fold składa się z 20 przypadków.
Powiedzmy, że posiadamy atrybut kolor oczu i pierwsze 20 przypadków to jedyne przypadki z niebieskim kolorem oczu.
W iteracji, w której te 20 przypadków będzie w foldzie przypadającym na zbiór testowy okaże się,
że w zbiorze treningowym nie będzie żadnego takiego przypadku i nasza trafność bardzo spadnie.

Z drugiej strony w innym przypadku może zdarzyć się odwrotna sytuacja, w której akurat kolor oczu niebieski
zawsze będzie prawidłowo klasyfikowany, choć próbki z inną wartością cechy wcale dobrze klasyfikowane być nie muszą.
Wtedy nasza trafność będzie zbytnio wysoka w tej iteracji.

Także warto włączać mieszanie, gdy dane są w jakiś sposób posortowane.
Jednak jeżeli dane są zależne czasowo, np. przez zmnieniajace się trendy to nie powinniśmy ich mieszać.
Dane uczące powinny być starsze w takim przypadku niż dane testowe.

''' 


# Zadanie 7

In [ ]:
'''
Parametr C odpowiada za wielkość marginesu błędu. Nie wpływa na sam sposób liczenia trafności ????.
Im większa wartość tym mniejszy margines i jest bardziej douczony do danych uczących. 
Parametr class_weight wpływa na wartość parametru C dla konkretnej klasy decyzyjnej. 
Czyli środek marginesu będzie bardziej lub mniej przesunięty w stronę którejś z klas decyzyjnych.
Wartość 'balanced' ustala wartość parametru class_weight odwrotnie proporcjonalnie do liczności klas.

balanced_accuracy nie jest przeskalowanym accuracy.
Jest to średnia z wartości sensitivity (recall) dla każdej klasy decyzyjnej ??? SPRAWDZIĆ DALEJ.

Czy mając zbiór 15k przypadków i 2 zrównoważone klasy, klasyfikator może z 10-krotnej CV 
uzyskiwać średnie balanced_accuracy<0.5? Jeśli tak, to w jakiej sytuacji?
Tak, może. Jeżeli mamy idealny klasyfikator, to recall dla każdej klasy wynosi 1.
Jeżeli mamy najgorszy klasyfikator, np. poprzez zamianę klas decyzyjnych perfekcyjengo klasyfikatora, to recall wynosi 0.
Z definicji balanced_accuracy to średnia wartości recall dla każdej klasy. Także w tym wypadku wyjdzie 0.
'''

# Zadanie 8

In [ ]:
# zero rule classifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score as gmean
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score
import time
data = pd.read_csv("./131767-svm.txt", sep='\t')
data[:3]
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': ['auto', 'scale'],
                     'C': [1, 1000],
                     'class_weight': ['balanced', None]},
                    {'kernel': ['linear'], 
                     'C': [1, 1000],
                    'class_weight': ['balanced', None]}]

# tuned_parameters = [{'kernel': ['rbf'], 
#                      'C': [1, 1000]}]


scores_list = ['accuracy', 'balanced_accuracy']
start = time.time()
for score in scores_list:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(random_state=42), 
                       tuned_parameters, 
                       scoring=score,
                       cv=10,
                      n_jobs=-1)
    
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
    
print(time.time()-start)

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'C': 1000, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}

Grid scores on development set:

0.816 (+/-0.023) for {'C': 1, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
0.766 (+/-0.030) for {'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}
0.841 (+/-0.025) for {'C': 1, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
0.788 (+/-0.011) for {'C': 1, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}
0.843 (+/-0.024) for {'C': 1000, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
0.835 (+/-0.016) for {'C': 1000, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}
0.843 (+/-0.024) for {'C': 1000, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
0.858 (+/-0.021) for {'C': 1000, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}
0.794 (+/-0.035) for {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}
0.81

In [ ]:
clf.cv_results_

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score as gmean
from skleanr.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
clf = SVC(random_state=42)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("balanced_accuracy_score: ", balanced_accuracy_score(y_test, y_pred, adjusted=False))
print(gmean(y_test, y_pred))

In [ ]:
# standarycja, bez użycia PCA
sca = StandardScaler()
X_train_std = sca.fit_transform(X_train)
X_test_std = sca.transform(X_test)

clf = LinearSVC(random_state=42)
clf.fit(X_train_std, y_train)


y_pred = clf.predict(X_test_std)
print(classification_report(y_test, y_pred))
print("balanced_accuracy_score: ", balanced_accuracy_score(y_test, y_pred, adjusted=False))

In [ ]:
# standaryzacja, PCA 3 wymiary
sca = StandardScaler()
X_train_std = sca.fit_transform(X_train)
X_test_std = sca.transform(X_test)

pca = PCA(n_components=119, random_state=42)
X_train_std_pca = pca.fit_transform(X_train_std)
X_test_std_pca = pca.transform(X_test_std)
print("explained_variance_ratio_: ", sum(pca.explained_variance_ratio_))

clf = SVC(random_state=42)
clf.fit(X_train_std_pca, y_train)


y_pred = clf.predict(X_test_std_pca)
print(classification_report(y_test, y_pred))
print("balanced_accuracy_score: ", balanced_accuracy_score(y_test, y_pred, adjusted=False))

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

In [ ]:
clf.cv_results_